In [ ]:
import Queue
import threading
import pysec.util.edgar_utilies as eu
import sym_to_ciks

from pysec.util import ftp_utilities


#import pysec.util.xbrl as xbrl

In [ ]:
TCKR = sym_to_ciks.sym_to_ciks['IBM']

In [ ]:
aho = eu.get_acc_table({'cik' : eu.short_to_long_cik(sym_to_ciks.sym_to_ciks['AIG']),
                        'qk'  : 'q'})


#0001326801
#0000051143

In [ ]:
pool = ftp_utilities.FTP_Pool(5)
q = Queue.Queue()
def get_xbrl_file(cik,acc):
    q.put({(cik,acc): pool.get(cik,acc)})

In [ ]:
th = []
for i in aho.Acc.values:
    print i
    t = threading.Thread(target = get_xbrl_file,
                         args=(str(TCKR),i))
    t.start()
    th.append(t)

for i in th: t.join()

while not q.empty():
    print q.get()

In [ ]:
str(TCKR)

In [ ]:
bb = lambda x : xbrl.XBRL(x)

In [ ]:
aho['xbrl'] = aho.xml.apply(bb)

In [ ]:
ftp.get_sec_XRBL(d['short_cik'],d['acc'])

In [ ]:
plt.plot_date(aho.time.values,y(aho.xbrl.values)/1000000000.0,'-')
plt.xlabel('Time',fontsize=20)
plt.ylabel('Cash in B$',fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

In [ ]:
y = lambda x : float(x.fields['Cash'])

In [ ]:
import numpy
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
y = numpy.vectorize(y)

In [ ]:
aho

In [ ]:
import matplotlib

In [ ]:
aho['Filing Date'].values

In [ ]:
import datetime

In [ ]:
datetime.datetime(aho['Filing Date'].values[0])

In [ ]:
import time

In [ ]:
w = time.strptime(aho['Filing Date'].values[0],"%Y-%m-%d")

In [ ]:
dt = datetime.datetime.fromtimestamp(time.mktime(time.strptime(aho['Filing Date'].values,"%Y-%m-%d")))

In [ ]:
dt

In [ ]:
gg = lambda x : datetime.datetime.fromtimestamp(time.mktime(time.strptime(x,"%Y-%m-%d")))

In [ ]:
aho['time'] = aho['Filing Date'].apply(gg)

In [ ]:
aho.xbrl.values[0].fields

In [ ]:
aho.to_dict

In [ ]:
aho

In [35]:
import Queue
import threading
import pysec.util.edgar_utilies as eu
import sym_to_ciks

from pysec.util import ftp_utilities

T = 'AET'

TCKR = sym_to_ciks.sym_to_ciks[T]

TCKR_df = eu.get_acc_table({'cik' : eu.short_to_long_cik(TCKR),
                            'qk'  : 'k'})

pool = ftp_utilities.FTP_Pool(4)
q    = Queue.Queue() # Return queue

def get_xbrl_file(cik,acc,q):
    pool.get(cik,acc,q) # Race condition failure

th = []
for i in TCKR_df.Acc.values:
    t = threading.Thread(target = get_xbrl_file,
                         args=(str(TCKR),i,q)) # Send queue as parameters to safeguard blocking
    t.start()
    th.append(t)

for t in th:
    t.join()

# add extra column to dataframe that just has "fileloc"
TCKR_df['Fileloc'] = ''

# Pull out of FIFO, put back into dataframe one by one
# will be in Quene in random order (or some function of filesize/avg internet speed)
while not q.empty():
    ret = q.get()
    TCKR_df.loc[TCKR_df['Acc'] == ret[0],'Fileloc'] = ret[1]
    

print TCKR_df

  Filings                   Acc Filing Date                Fileloc
0    10-K  0001122304-15-000038  2015-02-27  /tmp/aet-20141231.xml
1    10-K  0001122304-14-000037  2014-02-28  /tmp/aet-20131231.xml
2    10-K  0001122304-13-000037  2013-02-19  /tmp/aet-20121231.xml
3    10-K  0001122304-12-000045  2012-02-24  /tmp/aet-20111231.xml
4    10-K  0001122304-11-000032  2011-02-25  /tmp/aet-20101231.xml
5    10-K  0001122304-10-000024  2010-02-26  /tmp/aet-20091231.xml


In [36]:
import pysec.util.xbrl as xbrl

In [37]:
y = lambda x : xbrl.XBRL(x)
TCKR_df['xbrl']=TCKR_df['Fileloc'].apply(y)

0: Imputed NoncurrentAssets: 41638100000.0  from Assets(53402100000.0) == CurrentAssets(11764000000.0)+NoncurrentAssets(0)
0: Imputed NetIncomeAvailableToCommonStockholdersBasic: 232000000.0  from NetIncomeAttributableToParent(232000000.0) == NetIncomeAvailableToCommonStockholdersBasic(0)+PreferredStockDividendsAndOtherAdjustments(0)Z
0: Imputed IncomeFromContinuingOperationsAfterTax: 229800000.0  from NetIncomeLoss(229800000.0) == IncomeFromContinuingOperationsAfterTax(0)+IncomeFromDiscontinuedOperations(0)Z+ExtraordinaryItemsGainLoss(0)Z
0: Imputed OtherComprehensiveIncome: 0.0  from ComprehensiveIncome(229800000.0) == OtherComprehensiveIncome(0)+NetIncomeLoss(229800000.0)
0: Imputed IncomeFromEquityMethodInvestments: 0.0  from IncomeFromContinuingOperationsBeforeTax(428700000.0) == IncomeBeforeEquityMethodInvestments(428700000.0)+IncomeFromEquityMethodInvestments(0)
0: Imputed NonoperatingIncomeLossPlusInterestAndDebtExpense: 428700000.0  from NonoperatingIncomeLossPlusInterestAndDe

In [38]:
tenk_df = TCKR_df.copy()

In [39]:
tenq_df

,Filings,Acc,Filing Date,Fileloc,xbrl
0,10-Q,0001122304-15-000131,2015-08-04,/tmp/aet-20150630.xml,<pysec.util.xbrl.XBRL object at 0x10b0811d0>
1,10-Q,0001122304-15-000074,2015-04-28,/tmp/aet-20150331.xml,<pysec.util.xbrl.XBRL object at 0x10b081250>
2,10-Q,0001122304-14-000149,2014-10-28,/tmp/aet-20140930.xml,<pysec.util.xbrl.XBRL object at 0x10b0bfb10>
3,10-Q,0001122304-14-000126,2014-07-29,/tmp/aet-20140630.xml,<pysec.util.xbrl.XBRL object at 0x10b3ce3d0>
4,10-Q,0001122304-14-000067,2014-04-24,/tmp/aet-20140331.xml,<pysec.util.xbrl.XBRL object at 0x10b3ce150>
5,10-Q,0001122304-13-000138,2013-10-29,/tmp/aet-20130930.xml,<pysec.util.xbrl.XBRL object at 0x10b3ce2d0>
6,10-Q,0001122304-13-000123,2013-07-30,/tmp/aet-20130630.xml,<pysec.util.xbrl.XBRL object at 0x10b3ce690>
7,10-Q,0001122304-13-000067,2013-04-30,/tmp/aet-20130331.xml,<pysec.util.xbrl.XBRL object at 0x10b3cea50>
8,10-Q,0001122304-12-000143,2012-10-25,/tmp/aet-20120930.xml,<pysec.util.xbrl.XBRL object at 0x10b3ce090>
9,10-Q,0001122304-12-000122,2012-07-31,/tmp/aet-20120630.xml,<pysec.util.xbrl.XBRL object at 0x10b3ce410>


In [47]:
a = tenk_df.ix[1,'xbrl']

In [81]:
a.GetFactValue('us-gaap:CashAndCashEquivalentsAtCarryingValue','Instant')

1412300000.0

In [79]:
a.fields['Cash']

1412300000.0